In [9]:
import os
import numpy as np
import pytesseract
import cv2
import matplotlib.pyplot as plt
import json 

from utils import show, convert,  add_location, preprocessing_blur, read_templates, group_similar_x_and_command
# debug 
isDebugMode = False

def debug(msg):
    if isDebugMode:
        print(msg)

def debugShow(template):
    if isDebugMode:
        show(template)

# 디렉토리 경로
directory_path = "template_img/"
templates, files = read_templates(directory_path)

# 실행파일 위치저장
pytesseract.pytesseract.tesseract_cmd = "C:/Tesseract-OCR/tesseract.exe"

class OCR:
    def __init__(self):
        pass
 
    def get(self, img, mode):
        
        # debugShow(img)
        # 전처리 
        if mode == 'cmdStr':
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            _, roi = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
            
        elif mode == 'no':
            img = cv2.resize(img, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            # gau = cv2.GaussianBlur(gray, (3, 3), 0) // 흐리게 해보았지만 성능이 떨어짐
            _, roi = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

        elif mode == 'name':
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            roi = gray 
            
        # debugShow(roi)
        # 이미지 스캔          
        if mode == 'cmdStr':
            text = pytesseract.image_to_string(roi, lang='kor+eng --psm 6')
            return text
        elif mode == 'no':
            text = ''.join(filter(str.isdigit, pytesseract.image_to_string(roi, config='--psm 6')))
            return text
        elif mode == 'name':
            text = ''.join(filter(str.isalpha, pytesseract.image_to_string(roi, config='eng --oem')))
            return text      
        else:
            return "Invalid language type"
 


In [10]:
# 클래스 인스턴스 생성
ocr = OCR()

# 위치 정보를 추가
locationInfos = []
add_location(locationInfos, 'no', 290, 160, 330, 190)
add_location(locationInfos, 'cmdStr', 356, 168, 1050, 210)
add_location(locationInfos, 'name', 348, 135, 704, 169)
add_location(locationInfos, 'cmdImg', 356, 165, 1250, 210) 


In [11]:
## 메인 함수
def search(img_name): 

    # 이미지 읽기
    img = cv2.imread(img_name)

    # 이미지 있는지 체크 하고 없으면 그다음 이미지로 넘어감
    if img is None:
        print(f"이미지 {img_name} 을 찾을 수 없습니다.")
        return
    
    # 이미지 크기 확인
    # height, width, _ = img.shape

    # 결과값 dict
    results = {}

    # 위치 배열을 순회하며 이미지에서 자르고 텍스트 읽어오기
    for locationIndex, locationInfo in enumerate(locationInfos):
        name = locationInfo['name']
        # 이미지에서 특정 구간 자르기 
        img_roi = img[locationInfo['y1']:locationInfo['y2'], locationInfo['x1']:locationInfo['x2']]
        
        item = results.get(name, {})
        
        if name != 'cmdImg':
            item = ocr.get(img_roi, name) # OCR 실행   
            
        if name == 'cmdImg':
            roi_threshold = preprocessing_blur(img_roi) 
            debugShow(roi_threshold)
            
            # B1 템플릿 매칭 수행
            found_locations = []
            # debug("----------------------------------")
            for tidx, template in enumerate(templates):   
                
                # B1-1 템플릿 매칭 수행, 유사도 N% 이상인 위치 찾기
                result = cv2.matchTemplate(roi_threshold, template, cv2.TM_CCOEFF_NORMED)
                ratio = 0.90 
                matched = np.where(result >= ratio) 
    
                debug(f"템플릿 {tidx} {files[tidx]}  {result.max()}") # 최대 유사도 출력
                if len(matched[0]) > 0:
                    if( isDebugMode ) :
                        debugShow(template)
                        debug(f"founded list : {files[tidx]} {len(matched[0])}개")
                    # 찾은 위치와 유사도를 함께 저장
                    for pt in zip(*matched[::-1]):
                        data = { 'x': pt[0], 'y': pt[1], 'ratio': "{:.6f}".format(result[pt[1], pt[0]]), 'cmd': files[tidx] } 
                        found_locations.append(data)  
                        # debug(f">  {data}")
                
                # debug("----------------------------------")

            # 유사한 위치를 그룹핑, sort by x 
            item = group_similar_x_and_command(found_locations)
        
            debug(found_locations)
            if( isDebugMode ) :
                # 결과 이미지에 템플릿 위치 표시
                for found_location in found_locations:
                    print(found_location) 
                    h, w = template.shape 
                    x, y  = found_location['x'], found_location['y']
                    top_left = (x, y)
                    bottom_right = (x + w, y + h)
                    cv2.rectangle(img_roi, top_left, bottom_right, 255, 2)
 

            
        # 결과값 저장   
        print(item)
        results[name] = item
        
    # 결과값 출력 
    # print('결과')
    # print(results)
    # print(json.dumps(results,default=convert, indent=4, ensure_ascii=False))
 
    return results
                

In [12]:

# 이미지 폴더
img_dir = "target/raven/"

# 폴더안에 이미지 파일 리스트
img_names = [img_dir + f for f in os.listdir(img_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
# img_names = ["raven1.jpg", "raven2.jpg", "raven3.jpg", "raven4.jpg", "raven5.jpg", "raven6.jpg", "raven7.jpg", "raven8.jpg"]

print("이미지 리스트")
print(img_names)
res = []
# 이미지 순회 main() 함수 실행
for img_name in img_names:
    res.append(search(img_name))

filename = 'raven.json'
with open(filename, 'w', encoding='utf-8') as make_file:
    json.dump(res, make_file, default=convert,  ensure_ascii=False, indent="\t")


이미지 리스트
['target/raven/17.jpg', 'target/raven/18.jpg', 'target/raven/19.jpg', 'target/raven/20.jpg', 'target/raven/21.jpg', 'target/raven/22.jpg', 'target/raven/23.jpg', 'target/raven/24.jpg', 'target/raven/25.jpg', 'target/raven/26.jpg', 'target/raven/27.jpg', 'target/raven/28.jpg', 'target/raven/29.jpg', 'target/raven/30.jpg', 'target/raven/31.jpg', 'target/raven/32.jpg', 'target/raven/33.jpg', 'target/raven/34.jpg', 'target/raven/35.jpg', 'target/raven/36.jpg', 'target/raven/37.jpg', 'target/raven/38.jpg']
17

LeftRightComboBlackHole
[{'x': 5, 'y': 6, 'ratio': '0.975187', 'cmdImg': 'lp'}, {'x': 40, 'y': 6, 'ratio': '0.986876', 'cmdImg': 'rp'}, {'x': 75, 'y': 6, 'ratio': '0.986761', 'cmdImg': 'rk'}]
18

RightStraightLeftLowKick
[{'x': 5, 'y': 6, 'ratio': '0.956020', 'cmdImg': 'rp'}, {'x': 39, 'y': 6, 'ratio': '0.976239', 'cmdImg': 'lk'}]
19

PKCombo
[{'x': 6, 'y': 6, 'ratio': '0.964165', 'cmdImg': 'rp'}, {'x': 40, 'y': 6, 'ratio': '0.986564', 'cmdImg': 'rk'}]
20

ValkyrieLanceCombo
[